In [1]:
from toyaikit.chat import IPythonChatInterface

In [2]:
interface = IPythonChatInterface()

In [3]:
from pydantic_ai import Agent
from pydantic_ai.messages import TextPart, ToolCallPart, ToolReturnPart 

In [4]:
system_prompt = """
You are a helpful assistant. If you need input for tools arguments, ask the user, don't make it up.
"""

agent = Agent(
    'openai:gpt-4o-mini',
    instructions=system_prompt
)

In [5]:
@agent.tool_plain
def multiply(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

@agent.tool_plain
def dad_joke(name: str) -> str:
    """Tells a dad joke with the provided name."""
    return f"Dad and {name} walk into a bar but there are no empty tables."

In [6]:
from types import SimpleNamespace as D

In [9]:
result = await agent.run(user_prompt='hey', message_history=[])

In [14]:
for m in result.all_messages():
    for part in m.parts:
        break

In [18]:
t = type(part)

In [27]:
async def run(agent):
    message_history = []

    while True:
        user_input = interface.input()
        if user_input.lower() == "stop":
            interface.display("Chat ended.")
            break
    
        result = await agent.run(user_prompt=user_input, message_history=message_history)
    
        messages = result.new_messages()
        
        tool_calls = {}
        
        for m in messages:    
            for part in m.parts:
                kind = part.part_kind
                if kind == 'text':
                    interface.display_response(part.content)
                if kind == 'tool-call':
                    call_id = part.tool_call_id
                    tool_calls[call_id] = part
                if kind == 'tool-return':
                    call_id = part.tool_call_id
                    call = tool_calls[call_id]
                    result = part.content
                    interface.display_function_call(call.tool_name, call.args, result)
    
        message_history.extend(messages)

In [28]:
await run(agent)

You: tell me a joke


You: alexy


You: boring


You: no


You: stop


Chat ended.


## Coding agent

In [11]:
import os
import shutil

def copy_project_template(project_folder: str) -> str:
    """
    copy the template into the project folder

    arguments: project_folder - should be a lowercase string with underscores
    """
    if not project_folder:
        print("Project name cannot be empty.")
        return

    if os.path.exists(project_folder):
        print(f"Directory '{project_folder}' already exists. Please choose a different name or remove the existing directory.")
        return

    shutil.copytree('django_template', project_folder)
    print(f"Django template copied to '{project_folder}' directory.")

    return project_folder

In [13]:
project_folder = copy_project_template('todoist2')

Django template copied to 'todoist2' directory.


In [14]:
from pathlib import Path
import tools

agent_tools = tools.AgentTools(Path(project_folder))

In [15]:
coding_agent_tools_list = [
    agent_tools.execute_bash_command,
    agent_tools.read_file,
    agent_tools.search_in_files,
    agent_tools.see_file_tree,
    agent_tools.write_file
]

In [16]:
DEVELOPER_PROMPT = """
You are a coding agent. Your task is to modify the provided Django project template
according to user instructions. You don't tell the user what to do; you do it yourself using the 
available tools. First, think about the sequence of steps you will do, and then 
execute the sequence.
Always ensure changes are consistent with Django best practices and the project’s structure.

## Project Overview

The project is a Django 5.2.4 web application scaffolded with standard best practices. It uses:
- Python 3.8+
- Django 5.2.4 (as specified in pyproject.toml)
- uv for Python environment and dependency management
- SQLite as the default database (see settings.py)
- Standard Django apps and a custom app called myapp
- HTML templates for rendering views
- TailwindCSS for styling

## File Tree


├── .python-version
├── README.md
├── manage.py
├── pyproject.toml
├── uv.lock
├── myapp/
│   ├── __init__.py
│   ├── admin.py
│   ├── apps.py
│   ├── migrations/
│   │   └── __init__.py
│   ├── models.py
│   ├── templates/
│   │   └── home.html
│   ├── tests.py
│   └── views.py
├── myproject/
│   ├── __init__.py
│   ├── asgi.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── templates/
    └── base.html

## Content Description

- manage.py: Standard Django management script for running commands.
- README.md: Setup and run instructions, including use of uv for dependency management.
- pyproject.toml: Project metadata and dependencies (Django 5.2.4).
- uv.lock: Lock file for reproducible Python environments.
- .python-version: Specifies the Python version for the project.
- myapp/: Custom Django app with models, views, admin, tests, and a template (home.html).
  - migrations/: Contains migration files for database schema.
- myproject/: Django project configuration (settings, URLs, WSGI/ASGI entrypoints).
  - settings.py: Configures installed apps, middleware, database (SQLite), templates, etc.
- templates/: Project-level templates, including base.html.

You have full access to modify, add, or remove files and code within this structure using your available tools.


## Additional instructions

- Don't execute "runproject", but you can execute other commands to check if the project is working.
- Make sure you use Tailwind styles for making the result look beatiful
- Use pictograms and emojis when possible. Font-awesome is awailable
- Avoid putting complex logic to templates - do it on the server side when possible
"""

In [97]:
coding_agent = Agent(
    'openai:gpt-4o-mini',
    instructions=DEVELOPER_PROMPT,
    tools=coding_agent_tools_list
)

In [98]:
await run(coding_agent)

You: make a todo list


You: NoReverseMatch at / Reverse for 'complete_task' not found. 'complete_task' is not a valid view function or pattern name.


You:   File "C:\Users\alexe\git\workshops\coding-agent\todoist\myapp\urls.py", line 6, in <module>     path('complete/<int:task_id>/', complete_task, name='complete_task'),


You: add also delete button


You: stop


Chat ended.
